# Student Attendance, Academics and Discipline Final

In this Jupyter notebook, attendance data for students will be compiled that reflect the number of periods they were tardy as well as the number of periods they had an unexcused absence as well as the number of total absent periods. A total of discipline data logs is also calculated as well as the number of suspensions that they had. Finally, the academic data for students will be compiled by looking at their total academic GPA as well as the number of Ds and Fs that they are getting in classes currently.

In [1]:
import numpy as np
import pandas as pd

The Aeries SIS queries for the data that is needed is included as well as uploading them into the sheet.

In [2]:
# LIST STU SC SN ID ATT ATT.A0 ATT.A1 ATT.A2 ATT.A3 ATT.A4 ATT.A5 ATT.A6 ATT.A7 ATT.A8 ATT.A9 ATT.DT 
codes = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Codes.xlsx")

# LIST STU ID DIS DIS.CD? DIS.DT 
discipline = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Discipline.xlsx")

# LIST STU ADS DSP ADS.SC STU.ID ADS.DT ADS.CD DSP.DS DSP.DD
suspension = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Suspensions.xlsx")

# LIST STU SC ID STU.NM GR TP
gpa = pd.read_excel(r"C:\Users\derek.castleman\Desktop\GPA.xlsx")

# LIST STU ID FEE FEE.AM FEE.DT FEE.PD FEE.CD?
fees = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Fees.xlsx")

# LIST STU GBU GBK GBR STU.SC STU.ID STU.NM STU.GR GBK.PD GBK.NM GBR.CD GBU.CMK GBU.TG GBU.CSC
gradebook = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Gradebook.xlsx")

# LIST STU ID ACT ACT.CD? ACT.DT ACT.HR 
hours = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Hours.xlsx")

C:\Users\derek.castleman\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


KeyboardInterrupt: 

## Attendance

Attendance will be looked at in three different ways. First, the number of periods that a student was tardy to class will be summed up. I will follow this by calculating the number of periods that each student missed due to an unexcused absence. It will conclude by calculating the total periods that a student missed whether or not they were excused or unexcused.

In [ ]:
codes

In [ ]:
codes['Date']= pd.to_datetime(codes['Date'])
codes

In [ ]:
a = input('What is the start date you are interested in (mm/dd/yyyy):          ')
a

In [ ]:
b = input('What is the end date you are interested in (mm/dd/yyyy):          ')
b

In [ ]:
codes = codes[(codes['Date'] >= a) & (codes['Date'] <= b)]
codes

In [ ]:
codes = codes.melt(['School','Student#', 'Student ID', 'Date'], var_name='Period', value_name='Code')
codes

In [ ]:
totalabsent = codes[(codes['Code'] == 'A') | (codes['Code'] == 'Q') | 
                     (codes['Code'] == 'S') | (codes['Code'] == 'U') | (codes['Code'] == 'K') |
                      (codes['Code'] == 'I') | (codes['Code'] == 'M') | (codes['Code'] == 'X')]
totalabsent

In [ ]:
totalabsent['Periods Absent'] = 1
totalabsent

In [ ]:
totalabsent = totalabsent.groupby(by=['Student ID'])['Periods Absent'].sum().reset_index()
totalabsent

In [ ]:
unexcused = codes[(codes['Code'] == 'A') | (codes['Code'] == 'Q') | 
                     (codes['Code'] == 'S') | (codes['Code'] == 'U')]
unexcused

In [ ]:
unexcused['Unexcused Absent'] = 1
unexcused

In [ ]:
unexcused = unexcused.groupby(by=['Student ID'])['Unexcused Absent'].sum().reset_index()
unexcused

In [ ]:
totaltardy = codes[(codes['Code'] == 'C') | (codes['Code'] == 'D') | 
                     (codes['Code'] == 'T') | (codes['Code'] == 'Y') | (codes['Code'] == 'Z')]
totaltardy

In [ ]:
totaltardy['Periods Tardy'] = 1
totaltardy

In [ ]:
totaltardy = totaltardy.groupby(by=['Student ID'])['Periods Tardy'].sum().reset_index()
totaltardy

In [ ]:
attendance = pd.merge(totalabsent, unexcused, how='outer', on='Student ID')
attendance

In [ ]:
attendance = pd.merge(attendance, totaltardy, how='outer', on='Student ID')
attendance

## Discipline

A date will be input that will be selected for when discipline should be looked at beyond. A number of one will be assigned to each discipline log and then it will be summed for each of the students. The same thing will be done with the suspensions for a student.

In [ ]:
discipline

In [ ]:
discipline['Date']= pd.to_datetime(discipline['Date']) # Changing to proper date format
discipline

In [ ]:
discipline = discipline[(discipline['Date'] >= a) & (discipline['Date'] <= b)]
discipline

In [ ]:
discipline['Discipline Logs'] = 1 # Giving a value of one for each discipline
discipline

In [ ]:
# Summing up the discipline for each student
discipline = discipline.groupby(by=['Student ID'])['Discipline Logs'].sum().reset_index()
discipline

In [ ]:
suspension

In [ ]:
suspension = suspension[(suspension['Disposition'] == 'SUS-13') | (suspension['Disposition'] == '04')] # Selecting suspensions
suspension

In [ ]:
# Changing date into proper format
suspension['Disposition date']= pd.to_datetime(suspension['Disposition date'])
suspension

In [ ]:
# Selecting suspensions beyond the start 
suspension = suspension[(suspension['Disposition date'] >= a) & (suspension['Disposition date'] <= b)]
suspension

In [ ]:
suspension['Suspensions'] = 1 # Giving a value of 1 for each suspension
suspension

In [ ]:
# Summing up the suspensions for each student
suspension = suspension.groupby(by=['Student ID'])['Suspensions'].sum().reset_index()
suspension

In [ ]:
# Merging the two behavior columns together
behavior = pd.merge(discipline, suspension, how='outer', on='Student ID')
behavior

In [ ]:
behavior = behavior.fillna(0) # Filling in the missing values with zero
behavior

## Academics

The total academic GPA for students will be added to the attendance data along with any outstanding fees that they might have. The outstanding fees are calculated by the total they have been charged with the total that has been paid minused from it. The grades for the current term will be selected and any grade under a C- will be chosen with a sum for each student for how many Ds and Fs they have given.

In [ ]:
gpa

In [ ]:
# Adds the total GPA of each student with their attendance data
merge_one = pd.merge(gpa, attendance, how='left', on='Student ID')
merge_one

In [ ]:
final = pd.merge(merge_one, behavior, how='left', on='Student ID')
final

In [ ]:
final = final.fillna(0)
final

In [ ]:
gradebook

In [ ]:
z = input('Input term of interest (1, 2 etc.)        ') # Current term for grades is unput
z

In [ ]:
gradebook = gradebook[gradebook['Term'] == z] # Current term is selected from gradebook
gradebook

In [ ]:
# Failing grades are chosen
failing = gradebook[(gradebook['OverallMark'] == 'D+') | (gradebook['OverallMark'] == 'D') | 
                    (gradebook['OverallMark'] == 'D-') | (gradebook['OverallMark'] == 'F')]
failing

In [ ]:
# A value of 1 is given to each course that has been filtered
failing['D/F'] = 1
failing

In [ ]:
# Total number is summed up
failing = failing.groupby(by=['Student ID'])['D/F'].sum().reset_index()
failing

In [ ]:
final

In [ ]:
# Failing grades are added to all other merges
final = pd.merge(final, failing, on='Student ID', how='left')
final

In [ ]:
final = final.rename(columns={'D/F': "Q" + z + " D/F"})
final

## Fees and Community Service

The fees will be calculated for students by subtracting the amount paid from the amount charged. The community service hours will then be calculated by filtering over the date range and then summing the column for the hours. Everything will then be added to the dataframe to complete it.

In [ ]:
fees

In [ ]:
# Debts are calculated by subtracting amount paid from amount charged
fees['Fees/Debts'] = fees['Amount Chgd'] - fees['Amount Pd']
fees

In [ ]:
fees = fees[['Student ID', 'Fees/Debts']]
fees

In [ ]:
# Fees are added to the end
final = pd.merge(final, fees, on='Student ID', how='left')
final

In [ ]:
hours

In [ ]:
hours = hours[hours['Description_ACT_CD'] == 'Community Service']
hours

In [ ]:
hours['Date']= pd.to_datetime(hours['Date']) # Changes absent date to datetime
hours

In [ ]:
hours = hours[(hours['Date'] >= a) & (hours['Date'] <= b)]
hours

In [ ]:
hours = hours.groupby(by=['Student ID'])['Hours'].sum().reset_index()
hours

In [ ]:
final = pd.merge(final, hours, on='Student ID', how='left')
final

In [ ]:
# Any remaining nan are filled in with 0
final = final.fillna(0)
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Attendance and Discipline", filename = "Attendance and Discipline"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)